In [1]:
# create tensor files
import os, sys
import gensim
from gensim.models import KeyedVectors

eebo_models=list()
input_data = ["../models/eebo-henry_VII.w2v",
              "../models/eebo-henry_VIII.w2v",
              "../models/eebo-edward_VI.w2v",
              "../models/eebo-mary_I.w2v",
              "../models/eebo-elizabeth_I.w2v",
              "../models/eebo-james_I.w2v",
              "../models/eebo-charles_I.w2v",
              "../models/eebo-charles_II.w2v",
              "../models/eebo-oliver_cromwell.w2v"]

for fp in input_data:
    model_name = os.path.basename(fp).split(".")[0]
    print("starting: {0}".format(model_name))
    eebo_models.append([model_name,KeyedVectors.load(fp,mmap='r')])

starting: eebo-henry_VII
starting: eebo-henry_VIII
starting: eebo-edward_VI
starting: eebo-mary_I
starting: eebo-elizabeth_I
starting: eebo-james_I
starting: eebo-charles_I
starting: eebo-charles_II
starting: eebo-oliver_cromwell


In [4]:
outfilejson = "../tensors/tensors_all.json"

json_data = dict()
json_data['embeddings'] = list()

# limit model
key_terms = ["religion","church","self","fate","reformation",
             "elect","protestant","chosen","faith",
             "predestination"]

for m in eebo_models:
    print(m[0])
    
    tensor_filename = "../tensors/" + m[0] 
    outfiletsv = tensor_filename + '_tensor.tsv'
    outfiletsvmeta = tensor_filename + '_metadata.tsv'
    
    vectors=list()
    vocab=list()
    for t in key_terms:
        if t in m[1].vocab:
            nn = [x[0] for x in m[1].most_similar(t,topn=100)]
            for n in nn:
                vocab.append(n)
    vocab = set(vocab)
    
    with open(outfiletsv, 'wt') as file_vector:
        with open(outfiletsvmeta, 'wt',encoding='utf-8') as file_metadata:
            for word in vocab:
                file_metadata.write(word + '\n')
                vector_row = '\t'.join(str(x) for x in m[1][word])
                file_vector.write(vector_row + '\n')
                
    shape = [1000,50]
    json_data['embeddings'].append({
        'tensorName': str(m[0]),
        'tensorPath': 'https://raw.githubusercontent.com/catherinedparnell/Cultural-Analytics-IS/master/tensors/' + m[0] + '_tensor.tsv',
        'metadataPath': 'https://raw.githubusercontent.com/catherinedparnell/Cultural-Analytics-IS/master/tensors/' + m[0] + '_metadata.tsv',
        'tensorShape' : shape
    })
                

eebo-henry_VII
eebo-henry_VIII
eebo-edward_VI
eebo-mary_I
eebo-elizabeth_I
eebo-james_I
eebo-charles_I
eebo-charles_II
eebo-oliver_cromwell


In [5]:
import json
with open(outfilejson, 'wt') as fp:
    json.dump(json_data, fp)